In [ ]:
# Run this first, Comment it, restart session and run all.
# !pip install accelerate -U

In [ ]:
# #To connect google drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
OUT_DIR = '/content/test'

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split
seed=22

In [ ]:
data = pd.read_csv('./rt_reviews_data_2k.csv')

data=data.drop(['Prompt','title'], axis=1)
data=data.dropna()

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

import glob

tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')


In [ ]:
def summarize_text(text, model, tokenizer, max_length=800, num_beams=5):
    # Preprocess the text
    inputs = tokenizer.encode(
        "summarize: " + text,
        return_tensors='pt',
        max_length=max_length,
        truncation=True
    )

    # Generate the summary
    summary_ids = model.generate(
        inputs,
        max_length=150,
        num_beams=num_beams,
        # early_stopping=True,
    )

    # Decode and return the summary
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
# !pip install -U transformers
# !pip install -U datasets
# !pip install tensorboard
# !pip install sentencepiece
# !pip install accelerate
# !pip install evaluate
# !pip install rouge_score

In [ ]:
import torch
import pprint
import evaluate
import numpy as np

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
import datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['reviewText'],data['summary'],test_size=0.2, shuffle=True, random_state=seed)

dataset_train={'reviewText':X_train, 'summary':y_train}
dataset_test={'reviewText':X_test, 'summary':y_test}

In [ ]:
dataset_train=datasets.Dataset.from_pandas(pd.DataFrame(data=dataset_train))
dataset_test=datasets.Dataset.from_pandas(pd.DataFrame(data=dataset_test))

In [ ]:
MODEL = 't5-base'
BATCH_SIZE = 4
NUM_PROCS = 4
EPOCHS = 10
MAX_LENGTH = 800

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(MODEL)

# Function to convert text data into model inputs and targets
def preprocess_function(examples):
    inputs = [f"summarize: {article}" for article in examples['reviewText']]
    model_inputs = tokenizer(
        inputs,
        max_length=MAX_LENGTH,
        truncation=True,
        padding='max_length'
    )

    # Set up the tokenizer for targets
    targets = [summary for summary in examples['summary']]
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=MAX_LENGTH,
            truncation=True,
            padding='max_length'
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the function to the whole dataset
tokenized_train = dataset_train.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)
tokenized_test = dataset_test.map(
    preprocess_function,
    batched=True,
    num_proc=NUM_PROCS
)

In [ ]:
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

In [ ]:
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions[0], eval_pred.label_ids

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True,
        rouge_types=[
            'rouge1',
            'rouge2',
            'rougeL'
        ]
    )

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
training_args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=OUT_DIR,
    logging_steps=10,
    evaluation_strategy='steps',
    eval_steps=200,
    save_strategy='epoch',
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    dataloader_num_workers=4,
        eval_accumulation_steps = 1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
    compute_metrics=compute_metrics
)


In [ ]:
history = trainer.train()

In [ ]:
model_path = f"{OUT_DIR}/checkpoint-2000"  # the path where you saved your model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [ ]:
# !huggingface-cli login

In [ ]:
# model.push_to_hub("movie_reviews_trained_t5_checkpoint_2000_2024_03_29")